# Objetivo

Desta vez, a estratégia é treinar um pouco mais de Data Cleaning. Para isso, utilizarei a base de dados Coffee_Data e biblioteca, que contém mais de 10,000 linhas com dados fictícios, propositalmente "sujo".

## Importando a biblioteca e lendo a base de dados

In [1]:
import pandas as pd
import numpy as np
#numpy utilizado para tratar mudanças de tipo de colunas

In [2]:
df = pd.read_csv(r'C:\Desktop\Mentoria\Projetos\Coffee for Data Cleaning Training\Database\cafe_sales.csv')
df

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
...,...,...,...,...,...,...,...,...
9995,TXN_7672686,Coffee,2,2.0,4.0,NaN,UNKNOWN,2023-08-30
9996,TXN_9659401,NaN,3,NaN,3.0,Digital Wallet,NaN,2023-06-02
9997,TXN_5255387,Coffee,4,2.0,8.0,Digital Wallet,NaN,2023-03-02
9998,TXN_7695629,Cookie,3,NaN,3.0,Digital Wallet,NaN,2023-12-02


## Análise dos dados

_A primeira vista, é notório alguns valores com erro, NaN..._

_Para mais detalhes, utilizarei as funções de describe() e info()_

In [3]:
df.describe()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_1961373,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


_É perceptível a diferença de "Null's" entre as colunas_

_Também, todas as colunas como "Object"?_

_Primeiro, vamos olhar quantos valores vazios temos:_

## Conferindo Valores Vazios

In [5]:
n_columns = df.shape[0]
pctg_missing_data = df.isnull().sum()/n_columns *100
print(pctg_missing_data)

Transaction ID       0.00
Item                 3.33
Quantity             1.38
Price Per Unit       1.79
Total Spent          1.73
Payment Method      25.79
Location            32.65
Transaction Date     1.59
dtype: float64


_Como já mostrado na função describe, é possível inferir que a coluna "Transaction ID" não tem valores vazios._

_Já as demais colunas, todas tem valores nulos, com destaque a "Payment Method" e "Location", ambas com mais de 25% de seus dados faltando_

_Começarei a limpeza pelas colunas de "Item" e "Price Per Unit"_

## Colunas "Item" e "Price Per Unit"

In [6]:
df_unique_pairs = df[['Item', 'Price Per Unit']].drop_duplicates()
print(df_unique_pairs)

          Item Price Per Unit
0       Coffee            2.0
1         Cake            3.0
2       Cookie            1.0
3        Salad            5.0
5     Smoothie            4.0
6      UNKNOWN            3.0
7     Sandwich            4.0
8          NaN            3.0
14       ERROR            1.5
17       Juice            3.0
30         NaN            2.0
31     UNKNOWN            1.0
42         Tea            1.5
52     UNKNOWN            5.0
56        Cake            NaN
65    Sandwich            NaN
68       Salad          ERROR
69       ERROR            3.0
72         NaN            1.0
85         Tea            NaN
88       ERROR            5.0
100        NaN            5.0
104      Juice            NaN
118      ERROR            NaN
140       Cake        UNKNOWN
147      Salad        UNKNOWN
151        NaN            NaN
161      Juice        UNKNOWN
162     Cookie        UNKNOWN
165    UNKNOWN            4.0
168        NaN            4.0
186   Smoothie            NaN
196       

_Corrigirei os itens que estão sem valor e também, como não temos informação sobre os itens "Unknown", excluirei-os_

In [7]:
item_preco = {'Coffee' : 2.0, 
               'Tea' : 1.5,
               'Sandwich' : 4.0,
               'Salad' : 3.0,
               'Cake' : 3.0,
               'Cookie': 1.0,
               'Smoothie': 4.0,
               'Juice': 3.0}

#Invertendo o dicionário, para mais a frente, conseguir converter valores
preco_item = {price: item for item, price in item_preco.items()} 

#Convertendo a coluna Price Per Unit em "float"

df['Price Per Unit'] = df['Price Per Unit'].replace(['ERROR', 'UNKNOWN'], np.nan)
    #Na linha acima, convertemos ERROR e UNKNOWN em NaN

df['Price Per Unit'] = df['Price Per Unit'].astype(float)

#Atribuindo os respectivos valores dos produtos
df['Item'] = df['Item'].fillna(df['Price Per Unit'].map(preco_item))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    10000 non-null  object 
 1   Item              9939 non-null   object 
 2   Quantity          9862 non-null   object 
 3   Price Per Unit    9467 non-null   float64
 4   Total Spent       9827 non-null   object 
 5   Payment Method    7421 non-null   object 
 6   Location          6735 non-null   object 
 7   Transaction Date  9841 non-null   object 
dtypes: float64(1), object(7)
memory usage: 625.1+ KB


_Ainda é possível notar na coluna "Item" 61 valores restantes para serem tratados_

_Isso se deve ao fato das colunas "Item" e "Price Per Unit" ainda estarem com valores NaN_

_Uma forma de resolver a coluna "Price Per Unit" é dividir o total da compra ("Total Spent") pela quantidade comprada ("Quantity"), como solucionarei abaixo:_

In [8]:
df['Total Spent'] = df['Total Spent'].replace(['ERROR', 'UNKNOWN'], np.nan)
df['Total Spent'] = df['Total Spent'].astype(float)

#Para efetuar a conta, primeiro transformei a coluna em Float.
#O mesmo farei para a coluna Quantity

In [9]:
df['Quantity'] = df['Quantity'].replace(['ERROR', 'UNKNOWN'], np.nan)
df['Quantity'] = df['Quantity'].astype(float)

In [10]:
df['Price Per Unit'] = df['Price Per Unit'].fillna(df['Total Spent']/df['Quantity'])
df['Price Per Unit'] = df['Price Per Unit'].fillna(df['Price Per Unit'].map(preco_item))

C:\Users\021204631\AppData\Local\Temp\ipykernel_21108\3058530004.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Price Per Unit'] = df['Price Per Unit'].fillna(df['Price Per Unit'].map(preco_item))


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    10000 non-null  object 
 1   Item              9939 non-null   object 
 2   Quantity          9521 non-null   float64
 3   Price Per Unit    9962 non-null   float64
 4   Total Spent       9498 non-null   float64
 5   Payment Method    7421 non-null   object 
 6   Location          6735 non-null   object 
 7   Transaction Date  9841 non-null   object 
dtypes: float64(3), object(5)
memory usage: 625.1+ KB


_Como já não conseguimos estimar qual foram os outros itens comprados, vamos deleta-los utilizando drop.na()_

In [12]:
df.dropna(subset=['Item'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9939 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    9939 non-null   object 
 1   Item              9939 non-null   object 
 2   Quantity          9463 non-null   float64
 3   Price Per Unit    9903 non-null   float64
 4   Total Spent       9439 non-null   float64
 5   Payment Method    7374 non-null   object 
 6   Location          6698 non-null   object 
 7   Transaction Date  9780 non-null   object 
dtypes: float64(3), object(5)
memory usage: 698.8+ KB


_A coluna "Price Per Unit" ainda tem valores nulos, vamos trata-los:_

In [13]:
null_price = df[df['Price Per Unit'].isna()]
null_price

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
65,TXN_4987129,Sandwich,3.0,NaN,NaN,NaN,In-store,2023-10-20
629,TXN_9289174,Cake,NaN,NaN,12.0,Digital Wallet,In-store,2023-12-30
912,TXN_1575608,Sandwich,NaN,NaN,20.0,ERROR,Takeaway,2023-01-05
1008,TXN_7225428,Tea,NaN,NaN,3.0,Credit Card,Takeaway,2023-03-07
1436,TXN_7590801,Tea,NaN,NaN,6.0,Cash,Takeaway,ERROR
1482,TXN_3593060,Smoothie,NaN,NaN,16.0,Cash,NaN,2023-03-05
1674,TXN_9367492,Tea,2.0,NaN,NaN,Cash,In-store,2023-06-19
2229,TXN_8498613,Sandwich,2.0,NaN,NaN,NaN,NaN,2023-11-08
2289,TXN_7524977,UNKNOWN,4.0,NaN,NaN,ERROR,NaN,2023-12-09
2330,TXN_3849488,Salad,NaN,NaN,5.0,NaN,In-store,2023-03-01


_na coluna "Item" há UNKNOWN e algumas datas com erro... estas nos trataremos a seguir. Já os itens, como não é possivel determinar-los, droparei-os_

In [14]:
df= df[~df['Item'].isin(['ERROR', 'UNKNOWN'])]

In [15]:
#Filtro UNKNOWN para verificar se realmente foram deletados:
filter_item = df[df['Item'] == "UNKNOWN"]
filter_item

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date


In [16]:
df['Price Per Unit'] = df['Price Per Unit'].fillna(df['Item'].map(item_preco))
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9303 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    9303 non-null   object 
 1   Item              9303 non-null   object 
 2   Quantity          8868 non-null   float64
 3   Price Per Unit    9303 non-null   float64
 4   Total Spent       8842 non-null   float64
 5   Payment Method    6905 non-null   object 
 6   Location          6278 non-null   object 
 7   Transaction Date  9154 non-null   object 
dtypes: float64(3), object(5)
memory usage: 654.1+ KB


C:\Users\021204631\AppData\Local\Temp\ipykernel_21108\3954123981.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price Per Unit'] = df['Price Per Unit'].fillna(df['Item'].map(item_preco))


In [17]:
null_price

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
65,TXN_4987129,Sandwich,3.0,NaN,NaN,NaN,In-store,2023-10-20
629,TXN_9289174,Cake,NaN,NaN,12.0,Digital Wallet,In-store,2023-12-30
912,TXN_1575608,Sandwich,NaN,NaN,20.0,ERROR,Takeaway,2023-01-05
1008,TXN_7225428,Tea,NaN,NaN,3.0,Credit Card,Takeaway,2023-03-07
1436,TXN_7590801,Tea,NaN,NaN,6.0,Cash,Takeaway,ERROR
1482,TXN_3593060,Smoothie,NaN,NaN,16.0,Cash,NaN,2023-03-05
1674,TXN_9367492,Tea,2.0,NaN,NaN,Cash,In-store,2023-06-19
2229,TXN_8498613,Sandwich,2.0,NaN,NaN,NaN,NaN,2023-11-08
2289,TXN_7524977,UNKNOWN,4.0,NaN,NaN,ERROR,NaN,2023-12-09
2330,TXN_3849488,Salad,NaN,NaN,5.0,NaN,In-store,2023-03-01


## Coluna "Transaction Date"

_Uma coluna importantíssima para a análise é a "Transaction Date", pois sem ela, não é possível estimar o fluxo de pessoas por dia_

_Iniciarei por transforma-la em data_

In [18]:
df['Transaction Date']

0       2023-09-08
1       2023-05-16
2       2023-07-19
3       2023-04-27
4       2023-06-11
           ...    
9993    2023-10-20
9995    2023-08-30
9997    2023-03-02
9998    2023-12-02
9999    2023-11-07
Name: Transaction Date, Length: 9303, dtype: object

In [19]:
df.dropna(subset=['Transaction Date'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9154 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    9154 non-null   object 
 1   Item              9154 non-null   object 
 2   Quantity          8728 non-null   float64
 3   Price Per Unit    9154 non-null   float64
 4   Total Spent       8702 non-null   float64
 5   Payment Method    6802 non-null   object 
 6   Location          6178 non-null   object 
 7   Transaction Date  9154 non-null   object 
dtypes: float64(3), object(5)
memory usage: 643.6+ KB


C:\Users\021204631\AppData\Local\Temp\ipykernel_21108\2907906067.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['Transaction Date'], inplace=True)


In [20]:
df= df[~df['Transaction Date'].isin(['ERROR', 'UNKNOWN'])]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8869 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    8869 non-null   object 
 1   Item              8869 non-null   object 
 2   Quantity          8457 non-null   float64
 3   Price Per Unit    8869 non-null   float64
 4   Total Spent       8433 non-null   float64
 5   Payment Method    6603 non-null   object 
 6   Location          5976 non-null   object 
 7   Transaction Date  8869 non-null   object 
dtypes: float64(3), object(5)
memory usage: 623.6+ KB


## Coluna Quantity

In [21]:
null_quantity= df[df['Quantity'].isna()]
null_quantity

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
20,TXN_3522028,Smoothie,NaN,4.0,20.0,Cash,In-store,2023-04-04
55,TXN_5522862,Cookie,NaN,1.0,2.0,Credit Card,Takeaway,2023-03-19
57,TXN_2080895,Cake,NaN,3.0,3.0,Digital Wallet,In-store,2023-04-19
66,TXN_8501819,Juice,NaN,3.0,6.0,Cash,NaN,2023-03-30
117,TXN_2148617,Juice,NaN,3.0,9.0,Digital Wallet,UNKNOWN,2023-01-10
...,...,...,...,...,...,...,...,...
9932,TXN_8502079,Tea,NaN,1.5,3.0,Cash,NaN,2023-04-20
9935,TXN_9778251,Tea,NaN,1.5,6.0,NaN,Takeaway,2023-11-09
9944,TXN_7495283,Cake,NaN,3.0,15.0,Credit Card,Takeaway,2023-04-14
9957,TXN_6487003,Coffee,NaN,2.0,8.0,Credit Card,Takeaway,2023-11-15


_Usaremos a mesma intuição da coluna "Price Per Unit", onde dividimos "Total Spent" pela "Quantidade", mas agora, invertendo o raciocínio:_

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8869 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    8869 non-null   object 
 1   Item              8869 non-null   object 
 2   Quantity          8457 non-null   float64
 3   Price Per Unit    8869 non-null   float64
 4   Total Spent       8433 non-null   float64
 5   Payment Method    6603 non-null   object 
 6   Location          5976 non-null   object 
 7   Transaction Date  8869 non-null   object 
dtypes: float64(3), object(5)
memory usage: 623.6+ KB


In [23]:
df['Quantity'] = df['Quantity'].fillna(df['Total Spent']/df['Price Per Unit'])

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8869 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    8869 non-null   object 
 1   Item              8869 non-null   object 
 2   Quantity          8849 non-null   float64
 3   Price Per Unit    8869 non-null   float64
 4   Total Spent       8433 non-null   float64
 5   Payment Method    6603 non-null   object 
 6   Location          5976 non-null   object 
 7   Transaction Date  8869 non-null   object 
dtypes: float64(3), object(5)
memory usage: 623.6+ KB


In [25]:
null_quantity= df[df['Quantity'].isna()]
null_quantity

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
236,TXN_8562645,Salad,NaN,5.0,NaN,NaN,In-store,2023-05-18
278,TXN_3229409,Juice,NaN,3.0,NaN,Cash,Takeaway,2023-04-15
641,TXN_2962976,Juice,NaN,3.0,NaN,NaN,NaN,2023-03-17
738,TXN_8696094,Sandwich,NaN,4.0,NaN,NaN,Takeaway,2023-05-14
2796,TXN_9188692,Cake,NaN,3.0,NaN,Credit Card,NaN,2023-12-01
3203,TXN_4565754,Smoothie,NaN,4.0,NaN,Digital Wallet,Takeaway,2023-10-06
3224,TXN_6297232,Coffee,NaN,2.0,NaN,NaN,NaN,2023-04-07
3401,TXN_3251829,Tea,NaN,1.5,NaN,Digital Wallet,In-store,2023-07-25
4257,TXN_6470865,Coffee,NaN,2.0,NaN,Digital Wallet,Takeaway,2023-09-18
5841,TXN_5884081,Cookie,NaN,1.0,NaN,Digital Wallet,In-store,2023-07-05


_Para os casos que restaram, como não é possível estimar nem a quantidade comprada, nem o total gasto, droparei-os:_

In [26]:
df.dropna(subset='Quantity',inplace=True)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8849 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    8849 non-null   object 
 1   Item              8849 non-null   object 
 2   Quantity          8849 non-null   float64
 3   Price Per Unit    8849 non-null   float64
 4   Total Spent       8433 non-null   float64
 5   Payment Method    6590 non-null   object 
 6   Location          5961 non-null   object 
 7   Transaction Date  8849 non-null   object 
dtypes: float64(3), object(5)
memory usage: 622.2+ KB


## Coluna Total Spent

In [28]:
null_spent = df[df['Total Spent'].isna()]
null_spent

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
2,TXN_4271903,Cookie,4.0,1.0,NaN,Credit Card,In-store,2023-07-19
25,TXN_7958992,Smoothie,3.0,4.0,NaN,UNKNOWN,UNKNOWN,2023-12-13
42,TXN_6650263,Tea,2.0,1.5,NaN,NaN,Takeaway,2023-01-10
65,TXN_4987129,Sandwich,3.0,4.0,NaN,NaN,In-store,2023-10-20
94,TXN_6289610,Juice,3.0,3.0,NaN,Cash,Takeaway,2023-08-07
...,...,...,...,...,...,...,...,...
9890,TXN_2749289,Smoothie,2.0,4.0,NaN,Digital Wallet,Takeaway,2023-05-05
9893,TXN_3809533,Juice,2.0,3.0,NaN,Digital Wallet,Takeaway,2023-02-02
9954,TXN_1191659,Coffee,4.0,2.0,NaN,Credit Card,In-store,2023-11-21
9977,TXN_5548914,Juice,2.0,3.0,NaN,Digital Wallet,In-store,2023-11-04


_Para corrigir os NaN da coluna "Total Spent" é lógico: basta multiplcar a coluna "Quantity" pela coluna "Price Per Unit: "_

In [29]:
df['Total Spent'] = df['Total Spent'].fillna(df['Quantity'] * df['Price Per Unit'])

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8849 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    8849 non-null   object 
 1   Item              8849 non-null   object 
 2   Quantity          8849 non-null   float64
 3   Price Per Unit    8849 non-null   float64
 4   Total Spent       8849 non-null   float64
 5   Payment Method    6590 non-null   object 
 6   Location          5961 non-null   object 
 7   Transaction Date  8849 non-null   object 
dtypes: float64(3), object(5)
memory usage: 622.2+ KB
